In [1]:
import re
import pandas as pd
from tabulate import tabulate
from fetch_process_function import fetch_and_save_erc20_transfers, process_transactions, calculate_pnl, process_and_save_address
'''
data_process.ipynb和data_fetch.ipynb同样功能，但是通过封装函数实现了给日期地址等参数就可以返回数据，更加简洁
添加了pnl的计算，我们这里只计算第一种交易的pnl，其余的暂时忽略不计。
最后对于address也进行一个处理
'''
ADDRESS = "0x5be9a4959308A0D0c7bC0870E319314d8D957dBB"
# ADDRESS = "0x2c89a2ee92b9870f55989b4132a58c0e85222d86"
# ADDRESS = "0x6c2a355929ee1262305e385ad49b84fe5f5a4777"

START_DATE = "2024-09-25 08:00" # Singapore Time，the function will convert it to UTC
END_DATE = "2024-10-31 08:00"
OUTPUT_FILE = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result"
# OUTPUT_FILE = r"C:\Users\HUAWEI\Documents\GitHub\Project\etherscan\result"

# OUTPUT_FILE = r"C:\Users\HUAWEI\Documents\GitHub\Project\etherscan\address2"
# OUTPUT_FILE = r"C:\Users\HUAWEI\Documents\GitHub\Project\etherscan\address3"

API_KEY = "VQAIR728IM4Z8RZKPYBR4ESM5I3WBZK2C1"
BASE_TOKENS = {"USDT", "USDC", "USDE", "DAI"}
base_pattern = re.compile('|'.join(BASE_TOKENS))

In [2]:
fetch_and_save_erc20_transfers(
        address=ADDRESS,
        start_date=START_DATE,
        end_date=END_DATE,
        output_file=OUTPUT_FILE,
        offset=1000,
    )

transfers_df = pd.read_csv(OUTPUT_FILE + "/erc20_transfers.csv")
record_df_1, _, _ = process_transactions(transfers_df, OUTPUT_FILE, ADDRESS, base_pattern)

2025-02-06 21:00:27,250 - INFO - Start block: 20823901, End block: 21081895
2025-02-06 21:00:27,253 - INFO - Fetching ERC20 transfers from block 20823901 to 21081895...
2025-02-06 21:00:29,378 - INFO - Page 1: Retrieved 1000 transactions.
2025-02-06 21:00:31,465 - INFO - Page 2: Retrieved 1000 transactions.
2025-02-06 21:00:33,644 - INFO - Page 3: Retrieved 911 transactions.
2025-02-06 21:00:34,683 - WARNING - API request failed on page 4: Request failed: No transactions found
2025-02-06 21:00:34,718 - INFO - Final dataset contains 2911 unique transactions.
2025-02-06 21:00:34,912 - ERROR - An error occurred during fetch and save: Cannot save file into a non-existent directory: 'C:\Users\YuweiCao\Documents\GitHub\Project\Project\etherscan\result'


OSError: Cannot save file into a non-existent directory: 'C:\Users\YuweiCao\Documents\GitHub\Project\Project\etherscan\result'

In [ ]:
# 计算pnl
def process_pnl(record_df):
    if not record_df.empty:
        unique_tokens_count = record_df["token_symbol"].nunique()
        unique_tokens = record_df["token_symbol"].unique()
        results = []
        pnl_dfs = []  # 用于存储每个token的pnl_df
        # 在USDC USDE USDT DAI的价格默认为1：1的情况下，不考虑不同的稳定币作为单位的情况

        for token in unique_tokens:
            token_df = record_df[record_df["formatted_record"].str.contains(fr"\b{token}\b", regex=True)]
            unique_price_units = token_df["price_unit"].unique()
            price_units_str = ", ".join(unique_price_units)
            
            print(f"{token} (Price Units: {price_units_str}): {len(token_df)} records")

            pnl_df = calculate_pnl(token_df)
            print(tabulate(pnl_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))   
            total_pnl = pnl_df["pnl"].sum()

            pnl_dfs.append(pnl_df)  # 将每个token的pnl_df存储到列表中
            results.append({"token": token, "price_unit": price_units_str, "total_pnl": total_pnl})

        pnl_summary_df = pd.DataFrame(results)
        print(tabulate(pnl_summary_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))
        
        # 将结果保存为 CSV 文件
        pnl_summary_df.to_csv(f'pnl_summary.csv', index=False)
        print("PNL summary saved to pnl_summary.csv")

        # 如果需要将每个 token 的 PNL 详情也保存为 CSV
        for token in unique_tokens:
            token_df = record_df[record_df["formatted_record"].str.contains(fr"\b{token}\b", regex=True)]
            pnl_df = calculate_pnl(token_df)
            pnl_df.to_csv(f"pnl_details_{token}.csv", index=False)
            print(f"PNL details for {token} saved to pnl_details_{token}.csv")
    else:
        print("No transactions found")


process_pnl(record_df_1)

WETH (Price Units: aEthUSDT/WETH, USDT/WETH): 54 records
+---------------------+-------------+--------+----------+---------+---------+----------------+
| Date                |   timeStamp | Type   |   amount |   price |     pnl |   net_position |
+=====================+=============+========+==========+=========+=========+================+
| 2024-09-29 15:53:11 |  1727625191 | BUY    |    29.02 | 2669.39 | -122.05 |          29.02 |
+---------------------+-------------+--------+----------+---------+---------+----------------+
| 2024-09-29 15:53:23 |  1727625203 | BUY    |   108.72 | 2669.48 | -467.20 |         137.74 |
+---------------------+-------------+--------+----------+---------+---------+----------------+
| 2024-09-29 15:53:47 |  1727625227 | BUY    |    12.72 | 2669.67 |  -56.99 |         150.46 |
+---------------------+-------------+--------+----------+---------+---------+----------------+
| 2024-09-29 15:53:59 |  1727625239 | BUY    |   195.68 | 2669.76 | -894.27 |         34

In [ ]:
TOP_ACCOUNTS_DF = pd.read_csv(OUTPUT_FILE + "/top_accounts.csv")
transfers_df = pd.read_csv(OUTPUT_FILE + "/erc20_transfers.csv")
output_file = OUTPUT_FILE + "/address_counts_with_name_tags.csv"

process_and_save_address(transfers_df, TOP_ACCOUNTS_DF, output_file)

In [ ]:
address_counts = pd.read_csv(OUTPUT_FILE + "/address_counts_with_name_tags.csv")
address_threshold = 10
important_addresses = address_counts[address_counts["total_value_count"] > address_threshold]
important_addresses = important_addresses.sort_values(by="total_value_count", ascending=False)
# important_addresses = important_addresses[important_addresses["address"] != ADDRESS.lower()]

print(important_addresses)

In [ ]:
import sys
import os
import getpass
# find a way to use relative path, so that the function in dynamoUtil can be run in any machine
# src_path = r'C:\Users\YuweiCao\Documents\GitHub\char-python-common\src' 

# if src_path not in sys.path:
#     sys.path.append(src_path)

# USER = getpass.getuser()
# REPO_PATH = 'C:/Users/' + USER + '/Documents/GitHub/char-python-common'

# sys.path.append(REPO_PATH)
# sys.path.append(REPO_PATH + '/src')

USER = getpass.getuser()
REPO_PATH = 'C:/Users/' + USER + '/Documents/GitHub/char-python-common'

# 将路径添加到 sys.path
if REPO_PATH not in sys.path:
    sys.path.append(REPO_PATH)
if (REPO_PATH + '/src') not in sys.path:
    sys.path.append(REPO_PATH + '/src')

from dynamoUtil import price_bar_query
from datetime import datetime, timedelta

mapping = {
    'ETH': 'ETHUSDT-OKX-00000000-FUT',
    'SOL': 'SOLUSDT-OKX-00000000-FUT',
    'BTC': 'BTCUSDT-OKX-00000000-FUT',
}

def fetch_price_data(chain_name):
    today = datetime.now().replace(hour=23, minute=0, second=0, microsecond=0)
    start_date = today - timedelta(days=365) # 一年的数据

    today_str = today.strftime('%Y-%m-%d-%H-%M')
    start_date_str = start_date.strftime('%Y-%m-%d-%H-%M')
    
    instrument_code = mapping.get(chain_name)

    price_data = price_bar_query(instrument_code + '|1440', start_date_str, today_str, 'live')
    price_df = pd.DataFrame(price_data)
    print(f"Columns in DataFrame for {chain_name}: {price_df.columns}") 
    print(price_df.head())
    
    if price_df.empty:
        print(f"No valid data found for {chain_name}. Skipping.")
        return

    if 'timestamp' not in price_df.columns or 'close' not in price_df.columns:
        print(f"Required columns 'timestamp' or 'close' not found in the data for {chain_name}. Skipping.")
        return

    selected_columns = price_df[['timestamp', 'close']].copy()

    # selected_columns['date'] = pd.to_datetime(selected_columns['timestamp']).dt.date

    selected_columns.rename(columns={'close': 'price'}, inplace=True)
    
    # price_folder = r"C:\Users\YuweiCao\Documents\GitHub\Project\Project\etherscan\price\\"
    price_folder = r'C:\Users\HUAWEI\Documents\GitHub\Project\etherscan\price\\'

    selected_columns.to_csv(price_folder + f"{chain_name}_price.csv", index=False)
    
chain_names = mapping.keys()
for chain_name in chain_names:
    fetch_price_data(chain_name)

In [ ]:
import numpy as np

mapping = {
    'ETH': 'ETHUSDT-OKX-00000000-FUT',
    'SOL': 'SOLUSDT-OKX-00000000-FUT',
    'BTC': 'BTCUSDT-OKX-00000000-FUT',
}

def is_price_within_threshold(instrument, start_time, threshold):
    """
    判断最近三天的价格是否在阈值以内
    """
    primary_key_value = f"{instrument}|1"  # 假设使用 1440 分钟（日线）数据
    end_time = start_time - timedelta(days=1)
    price_data = price_bar_query(primary_key_value, start_time, end_time, env='live')

    if len(price_data) < 3:
        raise ValueError(f"价格数据不足三行，无法进行比较。")

    prices = price_data['close'].iloc[:3]

    # 计算价格差异平均值
    diff = np.mean(prices) -

    # 判断差异是否在阈值以内
    if diff <= threshold:
        return True
    else:
        return False

def classify_and_calculate_pnl(pnl_summary_df, mapping, price_bar_query, threshold=0.05, start_time=None, end_time=None):
    '''
    将 pnl_summary_df 中的 token 分为两类：主流币和其他币
    '''
    main_coin_results = []
    standalone_results = []

    for index, row in pnl_summary_df.iterrows():
        token = row["token"]
        price_unit = row["price_unit"]
        total_pnl = row["total_pnl"]
        
        # 判断是否含有中心币
        main_token = None
        for center_coin in mapping.keys():
            if re.search(fr"\b{center_coin}\b", token, re.IGNORECASE):
                main_token = center_coin
                break
        
        # 如果属于中心币，则判断 price_unit 是否正确
        if main_token:
            # 通过 price_bar_query 获取价格
            instrument = mapping[main_token]
            try:
                # 调用 price_bar_query 获取价格数据
                price_data = price_bar_query(instrument, start_time, end_time)
                if not price_data.empty:
                    # 获取对应的价格
                    actual_price = price_data["close"].iloc[-1]  # 获取最新价格
                    print(f"Actual price for {main_token}: {actual_price}")

                    # 判断 price_unit 是否包含主流币，并比较价格
                    if main_token in price_unit:
                        print(f"{token} matches {main_token} and price_unit is correct.")
                        main_coin_results.append({"token": main_token, "price_unit": price_unit, "total_pnl": total_pnl})
                    elif abs(actual_price - 1) <= threshold:  # 假设价格接近 1 表示稳定币
                        print(f"{token} matches {main_token} and price is within threshold. Adding to main coin results.")
                        main_coin_results.append({"token": main_token, "price_unit": price_unit, "total_pnl": total_pnl})
                    else:
                        print(f"{token} matches {main_token} but price or price_unit is incorrect.")
                        standalone_results.append({"token": token, "price_unit": price_unit, "total_pnl": total_pnl})
                else:
                    print(f"No price data found for {main_token}. Adding to standalone results.")
                    standalone_results.append({"token": token, "price_unit": price_unit, "total_pnl": total_pnl})
            except Exception as e:
                print(f"Error fetching price data for {main_token}: {e}")
                standalone_results.append({"token": token, "price_unit": price_unit, "total_pnl": total_pnl})
        else:
            print(f"{token} does not match any main token. Adding to standalone results.")
            standalone_results.append({"token": token, "price_unit": price_unit, "total_pnl": total_pnl})

    # 转换结果为 DataFrame
    main_coin_df = pd.DataFrame(main_coin_results)
    standalone_df = pd.DataFrame(standalone_results)

    # 打印汇总结果
    if not main_coin_df.empty:
        print("Main Coin PnL Results:")
        print(tabulate(main_coin_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))
    else:
        print("No Main Coin PnL Results Found.")
    
    if not standalone_df.empty:
        print("Standalone PnL Results:")
        print(tabulate(standalone_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))
    else:
        print("No Standalone PnL Results Found.")

    return main_coin_df, standalone_df
